<a href="https://colab.research.google.com/github/WSD2019-teamD/data_reduction/blob/master/colab/optimize_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 42.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=8fe58dfee6927bd121e88c279be21ddf0ec86875f93ba3984133a4b6b8987c02
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32040 sha256=95bdbf3723dd35051834a24f299a28c59e470b1c676270fdf8dfcbe04bb53652
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# NLP
import glob
from tqdm import tqdm
import math
import urllib
import gensim
from gensim import corpora
from collections import defaultdict

# pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

# Vis
from wordcloud import WordCloud
from PIL import Image
import matplotlib
matplotlib.use('Agg') 
import matplotlib.pylab as plt
font = {'family': 'TakaoGothic'}
matplotlib.rc('font', **font)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import gc
import logging
import pickle
from smart_open import open
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
import os
os.chdir('/content/drive/My Drive/Colab_Notebooks/data')
print(os.getcwd())

/content/drive/My Drive/Colab_Notebooks/data


In [3]:
ls

article/      corpus/      lda.model                  lda.model.state
cop.mm        dictionary/  lda.model.expElogbeta.npy  sim
cop.mm.index  dict.txt     lda.model.id2word


## 辞書を読み込み


In [12]:
if(os.path.exists("dictionary/dictionary.txt")):
  d = Dictionary.load_from_text("dictionary/dictionary.txt")
else:
  print("no dictionary")

no dictionary


## コーパスを読み込み


In [0]:
if(os.path.exists("corpus/corpus.pkl")):
  corpus = pickle.dump(corpus,f)
else:
  print("no corpus")

# トピック数の検討

モデルの評価指標として一般的に用いられる Perplexity と Coherence を用いて、トピック数を検討していこうと思います。

In [0]:
#Metrics for Topic Models
start = 2
limit = 22
step = 1

coherence_vals = []
perplexity_vals = []

for n_topic in tqdm(range(start, limit, step)):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=n_topic, random_state=0)
    perplexity_vals.append(np.exp2(-lda_model.log_perplexity(corpus)))
    coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=texts, dictionary=d, coherence='c_v')
    coherence_vals.append(coherence_model_lda.get_coherence())

In [0]:
# evaluation
x = range(start, limit, step)

fig, ax1 = plt.subplots(figsize=(12,5))

# coherence
c1 = 'darkturquoise'
ax1.plot(x, coherence_vals, 'o-', color=c1)
ax1.set_xlabel('Num Topics')
ax1.set_ylabel('Coherence', color=c1); ax1.tick_params('y', colors=c1)

# perplexity
c2 = 'slategray'
ax2 = ax1.twinx()
ax2.plot(x, perplexity_vals, 'o-', color=c2)
ax2.set_ylabel('Perplexity', color=c2); ax2.tick_params('y', colors=c2)

# Vis
ax1.set_xticks(x)
fig.tight_layout()
plt.show()

# save as png
plt.savefig('metrics.png') 